In [ ]:
# Import necessary modules of Rasa nlu
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# Create a trainer that uses this config
trainer = Trainer(config.load("config_spacy.yml"))

# Load the training data
training_data = load_data('train_data_demo.json')

# Create an interpreter by training the model
interpreter = trainer.train(training_data)


In [ ]:
# Import iexfinance
from iexfinance.stocks import Stock
from iexfinance.stocks import get_sector_performance
from iexfinance.stocks import get_ipo_calendar
# Import re 
import re

# Import random
import random

# Import string
import string

# Import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
# import mpl_finance
import mpl_finance as mpf
from matplotlib.pylab import date2num

# Import datetime
import datetime

# Import pandas
import pandas

# Import sqlite3
import sqlite3

In [ ]:
# Response

response_group = {
    # greeting and finish
    "greet":[
                "Hello! My name is Stockchatbot. It's a pleasure to help you.",
                "Nice to meet you. My name is Stockchatbot. I could help you to get stock information.",
               ],
    
    "finish":[
                "Bye. It is a pleasure to help you!",
                "Goodbye. I'm glad that I could help you!",
               ],
    "function_intro":["Currently I can help you with:\n 1. Get current price of some stocks \n 2. Get yesterday price of some stocks\n 3. Get open & close price, highest & lowest price of some stocks\n 4. Get history price and volume of some stocks with a candlestick chart in different forms\n 5. Get current sector performance and get stock information & stock price performance(including financial report) in specific sector\n 6. Recommend merit-based shares, developmental shares, speculative shares in different sector and get quote, candlestick abstract chart of these shares\n 7. Get the latest financial report of one stock\n 8. Get the relevant latest news of one stock\n 9. Get recent Initial Public Offerings (IPO)"],
    
    "phonenumber":[
                           "perfect, welcome back!",
                          ],

    "current_price":[
                           "The current price of {} is {}.",
                           "{} has a real-time price of {}.",
                          ], 
    
    "op_cl_price":[
                           "The open and close price of {} is {} & {}. The highest price is {}, the lowest price is {}.",
                       ], 
    
    "previous_price":[
                           "The specific price performance of {} is:\n date: {}\n open: {} close: {}\n high: {} low: {}\n volume: {}",
                           ], 
    
     "quote":[
                 "Stock {}:\ncompnay name is {}\nprimary Exchange is {}\nin the sector of {}\nopen price is {}\nclose price is {}\nhighest price is {}\nlowest price is {}\nopen time is {}\nclose time is {}\nlatest price is {}\ndelayed price is {}\ndelayed price time is {}\nextended price is {}\nextended price time is {}\nvolume is {}\naverage total volume is {}\nthe highest price in the latest 52 weeks is {}\nthe lowest price in the latest 52 weeks is {}",
                 ], 
    
     "ipo":[
             "Sorry for keeping you waiting!\n\ncompany: {}\nsymbol: {}\nprice: {}\nshares: {}\namount: {}\nfloat: {}\npercent: {}\nmarket: {}\nexpected time: {}\n\ncompany: {}\nsymbol: {}\nprice: {}\nshares: {}\namount: {}\nfloat: {}\npercent: {}\nmarket: {}\nexpected time: {}\n\ncompany: {}\nsymbol: {}\nprice: {}\nshares: {}\namount: {}\nfloat: {}\npercent: {}\nmarket: {}\nexpected time: {}\n\ncompany: {}\nsymbol: {}\nprice: {}\nshares: {}\namount: {}\nfloat: {}\npercent: {}\nmarket: {}\nexpected time: {}\n\ncompany: {}\nsymbol: {}\nprice: {}\nshares: {}\namount: {}\nfloat: {}\npercent: {}\nmarket: {}\nexpected time: {}\n\n",
             ], 
    
     "stock_search":[
                           "I'm sorry :( I couldn't find anything like that", 
                           "{} is a stock you are looking for!", 
                           "{} or {} would answer your question!", 
                           "{} is one option, I can tell you more :)",
                          ], 
    
     "news":[
                "date: {}\n\nheadline: {}\n\nsource: {}\n\nlink: {}\n\nsummary: {}",
                ], 
    
     "finreport":[
                     "report date: {}\ngross profit: {}\ncost of revenue: {}\noperating revenue: {}\ntotal revenue: {}\noperating income: {}\nnet income: {}\nresearch & development: {}\noperating expense: {}\ncurrent assets: {}\ntotal assets: {}\ntotal liabilities: {}\ncurrent cash: {}\ncurrent debt: {}\ntotal cash: {}\ntotal debt: {}\nshareholder equity: {}\ncash change: {}\ncash flow: {}\noperating gains losses: {}",
                    ], 




}

def respond_sentence(intent):
    if intent == "stock_search":
        return response_group["stock_search"] 
    else:
        return random.choice(response_group[intent])

In [ ]:
# Define the state

# Define the menu state
MENU = 0
# Define the authed state
AUTHED=1
# Define the choose item state
CHOOSE_ITEM = 2
# Define the current price state
CRT_PRICE = 3
# Define the open close price state
OP_CL_PRICE = 4
# Define the previous price state
PRE_PRICE = 5
# Define the month price state
MON_PRI = 6
# Define the month abstract price state
MON_ABS_PRI = 7
# Define the month price and volume state
MON_VOL_PRI = 8
# Define the sector stock list state
SEC_STO_LIS = 9
# Define the quote state
QUOTE = 10
# Define the ipo state
IPO = 11
# Define the stock_seaech state
STO_SEAR = 12
# Define the news state
NEWS = 13
# Define the financial report state
FINREPORT = 14


In [ ]:
# Define the policy rules

policy_rules = {  
    
    # menu
    (MENU, "greet"): (MENU, respond_sentence("greet"), None),
    (MENU, "function_intro"): (MENU, "you'll have to log in first, what's your phone number?", AUTHED),
    (MENU, "phonenumber"): (AUTHED, respond_sentence("phonenumber"), None),
    (AUTHED, "function_intro"): (CHOOSE_ITEM, respond_sentence("function_intro"), None),    
    
    # get stock information
    
    # get current price    
    (CHOOSE_ITEM, "current_price"): (CRT_PRICE, respond_sentence("current_price"), None),
    # get current price in several times
    (CRT_PRICE, "current_price"): (CRT_PRICE, respond_sentence("current_price"), None),
    # return to the menu
    (CRT_PRICE, "finish"): (CHOOSE_ITEM, "I could help you to find other stock information.", None),
    
    # get op_cl price    
    (CHOOSE_ITEM, "op_cl_price"): (OP_CL_PRICE, respond_sentence("op_cl_price"), None),
    # get op_cl price  
    (CRT_PRICE, "op_cl_price"): (OP_CL_PRICE, respond_sentence("op_cl_price"), None),
    # get current price in several times
    (OP_CL_PRICE, "op_cl_price"): (OP_CL_PRICE, respond_sentence("op_cl_price"), None),
    # return to the choose item
    (OP_CL_PRICE, "finish"): (CHOOSE_ITEM, "I could help you to find other stock information.", None),
    
    # get previous price  
    (CHOOSE_ITEM, "previous_price"): (PRE_PRICE, respond_sentence("previous_price"), None),
    # get op_cl price  
    (CRT_PRICE, "previous_price"): (PRE_PRICE, respond_sentence("previous_price"), None),
    # get op_cl price  
    (OP_CL_PRICE, "previous_price"): (PRE_PRICE, respond_sentence("previous_price"), None),
    # get previous price in several times
    (PRE_PRICE, "previous_price"): (PRE_PRICE, respond_sentence("previous_price"), None),
    # return to the choose item and ask if get the month price
    (PRE_PRICE, "finish"): (CHOOSE_ITEM, "I could help you to find month price of this stock in three forms: Candlestick Chart; Candlestick Abstract Chart; Candlestick and Volume Chart. Which one do you like?", None),
    
    # get month price    Candlestick Chart
    (CHOOSE_ITEM, "month_price"): (MON_PRI, "let's see it.", None),
    (CRT_PRICE, "month_price"): (MON_PRI, "let's see it.", None),
    (OP_CL_PRICE, "month_price"): (MON_PRI, "let's see it.", None),
    (PRE_PRICE, "month_price"): (MON_PRI, "let's see it.", None),
    (MON_PRI, "month_price"): (MON_PRI, "let's see it.", None),
    # return to the choose item
    (MON_PRI, "finish"): (CHOOSE_ITEM, "I could help you to find other stock information.", None),
    
    # get month abstract price    Candlestick Abstract Chart 
    (CHOOSE_ITEM, "month_abstract_price"): (MON_ABS_PRI, "let's see it.", None),
    (CRT_PRICE, "month_abstract_price"): (MON_ABS_PRI, "let's see it.", None),
    (OP_CL_PRICE, "month_abstract_price"): (MON_ABS_PRI, "let's see it.", None),
    (PRE_PRICE, "month_abstract_price"): (MON_ABS_PRI, "let's see it.", None),
    (MON_PRI, "month_abstract_price"): (MON_ABS_PRI, "let's see it.", None),
    # get month abstract price in several times
    (MON_ABS_PRI, "month_abstract_price"): (MON_ABS_PRI, "let's see it.", None),
    # return to the choose item
    (MON_ABS_PRI, "finish"): (CHOOSE_ITEM, "I could help you to find other stock information.", None),
    
    # get month price and volume    Candlestick and Volume Chart  
    (CHOOSE_ITEM, "month_volume_price"): (MON_VOL_PRI, "let's see it.", None),
    (CRT_PRICE, "month_volume_price"): (MON_VOL_PRI, "let's see it.", None),
    (OP_CL_PRICE, "month_volume_price"): (MON_VOL_PRI, "let's see it.", None),
    (PRE_PRICE, "month_volume_price"): (MON_VOL_PRI, "let's see it.", None),
    (MON_PRI, "month_volume_price"): (MON_VOL_PRI, "let's see it.", None),
    (MON_ABS_PRI, "month_volume_price"): (MON_VOL_PRI, "let's see it.", None),
    # get month volume price in several times
    (MON_VOL_PRI, "month_volume_price"): (MON_VOL_PRI, "let's see it.", None),
    # return to the choose item
    (MON_VOL_PRI, "finish"): (CHOOSE_ITEM, "I could help you to find other stock information.", None),
    
    # be asked the volume but send the month price and volume    Candlestick and Volume Chart  
    (MON_PRI, "volume"): (CHOOSE_ITEM, "Why don't you let me tell you the Candlestick and Volume Chart of it.", MON_VOL_PRI),
    (CHOOSE_ITEM, "volume"): (CHOOSE_ITEM, "Why don't you let me tell you the Candlestick and Volume Chart of it.", MON_VOL_PRI),
    (CHOOSE_ITEM, "affirm"): (CHOOSE_ITEM, "The chart is on the way.", None),  
    (MON_VOL_PRI, "volume"): (MON_VOL_PRI, "let's see it.", None),
    # return to the choose item
    (MON_VOL_PRI, "finish"): (CHOOSE_ITEM, "I could help you to find other stock information.", None),
    
    # get current sector performance   
    (CHOOSE_ITEM, "sectorperformance"): (CHOOSE_ITEM, "I could help you to find stock information and stock price performance(including financial report) in one of these sector. Which sector stock list do you want me to print for you?", None),
    (CHOOSE_ITEM, "stocklist"): (SEC_STO_LIS, "The list is here.I can tell you the quote, candlestick and volume chart, financial report of these stocks.", None),
    # get quote and candlestick and volume chart and financial report
    (SEC_STO_LIS, "finreport"): (FINREPORT, respond_sentence("finreport"), None),
    (SEC_STO_LIS, "month_volume_price"): (MON_VOL_PRI, "let's see it.", None),
    (SEC_STO_LIS, "quote"): (QUOTE, respond_sentence("quote"), None),
    (QUOTE, "month_volume_price"): (MON_VOL_PRI, "let's see it.", None),
    (QUOTE, "finreport"): (FINREPORT, respond_sentence("finreport"), None),
    (MON_VOL_PRI, "finreport"): (FINREPORT, respond_sentence("finreport"), None),
    (MON_VOL_PRI, "quote"): (QUOTE, respond_sentence("quote"), None),
    (FINREPORT, "month_volume_price"): (MON_VOL_PRI, "let's see it.", None), 
    (FINREPORT, "quote"): (QUOTE, respond_sentence("quote"), None),
    # return to the choose item
    (QUOTE, "finish"): (CHOOSE_ITEM, "I could help you to find other stock information.", None),
    (MON_VOL_PRI, "finish"): (CHOOSE_ITEM, "I could help you to find other stock information.", None),
    (FINREPORT, "finish"): (CHOOSE_ITEM, "I could help you to find other stock information.", None),
    
    # get recent IPO    
    (CHOOSE_ITEM, "ipo"): (IPO, respond_sentence("ipo"), None),
    # return to the choose item
    (IPO, "finish"): (CHOOSE_ITEM, "I could help you to find other stock information.", None),
    
    # be asked to recommend good stocks and change into the stock_search state
    (CHOOSE_ITEM, "goodstock"): (STO_SEAR, "Please tell me what kind of shares and which sector you want me to recommend.", None),
    (STO_SEAR, "stock_search"): (STO_SEAR, respond_sentence("stock_search"), None),
    (STO_SEAR, "sector"): (STO_SEAR, respond_sentence("stock_search"), None),
    # get quote and candlestick abstract chart
    (STO_SEAR, "quote"): (QUOTE, respond_sentence("quote"), None),
    (STO_SEAR, "month_abstract_price"): (MON_ABS_PRI, "let's see it.", None),
    (QUOTE, "month_abstract_price"): (MON_ABS_PRI, "let's see it.", None),
    (MON_ABS_PRI, "quote"): (QUOTE, respond_sentence("quote"), None),
    # return to the choose item
    (QUOTE, "finish"): (CHOOSE_ITEM, "I could help you to find other stock information.", None),
    (MON_ABS_PRI, "finish"): (CHOOSE_ITEM, "I could help you to find other stock information.", None),
    
    # get news of one stock 
    (CHOOSE_ITEM, "news"): (NEWS, respond_sentence("news"), None),
    (CRT_PRICE, "news"): (NEWS, respond_sentence("news"), None),
    (OP_CL_PRICE, "news"): (NEWS, respond_sentence("news"), None),
    (PRE_PRICE, "news"): (NEWS, respond_sentence("news"), None),
    (NEWS, "current_price"): (CRT_PRICE, respond_sentence("current_price"), None),
    (NEWS, "op_cl_price"): (OP_CL_PRICE, respond_sentence("op_cl_price"), None),
    (NEWS, "previous_price"): (PRE_PRICE, respond_sentence("previous_price"), None),
    # return to the choose item
    (NEWS, "finish"): (CHOOSE_ITEM, "I could help you to find other stock information.", None),
    
    # get financial report of one stock  
    (CHOOSE_ITEM, "finreport"): (FINREPORT, respond_sentence("finreport"), None),
    # return to the choose item
    (FINREPORT, "finish"): (CHOOSE_ITEM, "I could help you to find other stock information.", None),
    
    # return to the menu
    (CHOOSE_ITEM, "finish"): (MENU, respond_sentence("finish"), None), 
    
    
}

In [ ]:
# Define match_intent()
keywords = {'sectorperformance': ['sector performance', 'sectorperformance', 'Sectorperformance', 'Sector performance', 'Sector Performance']}
# Define a dictionary of patterns
patterns = {}
# Iterate over the keywords dictionary
for key, values in keywords.items():
    # Create regular expressions and compile them into pattern objects
    patterns[key] = re.compile('|'.join(values))

def match_intent(message):
    matched_intent = None
    for intent, pattern in patterns.items():
        # Check if the pattern occurs in the message 
        if pattern.search(message):
            matched_intent = intent
            return matched_intent
        else:
            return interpreter.parse(message)['intent']['name']

# Define match_entity()
def match_entity(message):
    
    # no entity
    if interpreter.parse(message)['entities'] == []:
        return []
    
    # match company name
    if interpreter.parse(message)['entities'][0]['entity'] == 'company':
        return interpreter.parse(message)['entities'][0]['value']
    # match sector name
    if interpreter.parse(message)['entities'][0]['entity'] == 'sector':
        return interpreter.parse(message)['entities'][0]['value']


# Define send_message 
def send_message(state, pending, message):
    # calculate the new_state, response, and pending_state
    new_state, response, pending_state = respond(state, message)
    # send_response by wxpy
    my_friend.send(response)
    if match_intent(message) == 'month_price':
        my_friend.send_image('monthprice.png')
    if match_intent(message) == 'month_abstract_price':
        my_friend.send_image('monthabstractprice.png')
    if match_intent(message) == 'month_volume_price':
        my_friend.send_image('monthvolumeprice.png')
    if match_intent(message) == 'sectorperformance':
        my_friend.send_image('sectorperformance.png')                
    # judge the pending
    if pending is not None:
        # do pending: calculate the new_state, response, and pending_state
        new_state, response, pending_state = policy_rules[pending]
        # send_response by wxpy 
        my_friend.send(response)
        if new_state == MON_VOL_PRI:
            my_friend.send_image('monthvolumeprice.png')
        # clear the pending
        if response is not None:
            return new_state, None
    # create the new pending    
    if pending_state is not None:
        pending = (pending_state, match_intent(message))
    return new_state, pending



# Define respond()
def respond(state, message):
    entity = match_entity(message)
    # calculate the new_state
    new_state = policy_rules[(state, match_intent(message))][0]
    # create the pending_state
    pending_state = policy_rules[(state, match_intent(message))][2]

    # greeting
    if match_intent(message) == 'greet':
        response = policy_rules[(state, match_intent(message))][1]
    # finish
    if match_intent(message) == 'finish':
        response = policy_rules[(state, match_intent(message))][1]  
    # affirm
    if match_intent(message) == 'affirm':
        response = policy_rules[(state, match_intent(message))][1]  
    # function_intro
    if match_intent(message) == 'function_intro':
        response = policy_rules[(state, match_intent(message))][1]
    # phonenumber      
    if match_intent(message) == 'phonenumber':
        response = policy_rules[(state, match_intent(message))][1]
    # get current price
    if match_intent(message) == 'current_price':
        response = policy_rules[(state, match_intent(message))][1].format(entity, get_current_price(entity))    
    # get current price
    if match_intent(message) == 'op_cl_price':
        response = policy_rules[(state, match_intent(message))][1].format(*get_op_cl(entity))    
    # get yesterday price
    if match_intent(message) == 'previous_price':
        response = policy_rules[(state, match_intent(message))][1].format(*get_previous_price(entity))        
    # get month price
    if match_intent(message) == 'month_price':
        response = policy_rules[(state, match_intent(message))][1]
        get_monthly_price(entity)    
    # get month abstract price
    if match_intent(message) == 'month_abstract_price':
        response = policy_rules[(state, match_intent(message))][1]
        get_monthly_abstract_price(entity)
    # get month price and volume
    if match_intent(message) == 'month_volume_price':
        response = policy_rules[(state, match_intent(message))][1]
        get_monthly_volume_price(entity)    
    # volume    
    if match_intent(message) == 'volume':
        response = policy_rules[(state, match_intent(message))][1]
        get_monthly_volume_price(entity)    
    # sectorperformance   
    if match_intent(message) == 'sectorperformance':
        response = policy_rules[(state, match_intent(message))][1]
        #get_sector_perform()
    # sectorstocklist   
    if match_intent(message) == 'stocklist' and entity == 'technology':
        response = policy_rules[(state, match_intent(message))][1]
        my_friend.send_file('technologystocklist.pdf')
    # sectorstocklist   
    if match_intent(message) == 'stocklist' and entity == 'energy':
        response = policy_rules[(state, match_intent(message))][1]
        my_friend.send_file('energystocklist.pdf')
    # quote
    if match_intent(message) == 'quote':
        response = policy_rules[(state, match_intent(message))][1].format(*get_quote(entity))   
    # ipo
    if match_intent(message) == 'ipo':
        response = policy_rules[(state, match_intent(message))][1].format(*get_ipo_info()[0], *get_ipo_info()[1], *get_ipo_info()[2], *get_ipo_info()[3], *get_ipo_info()[4])
    # goodstock     
    if match_intent(message) == 'goodstock':
        response = policy_rules[(state, match_intent(message))][1]    
    # stock_search
    if match_intent(message) == 'stock_search':
        number, sharenames = respond_STO_SEAR(message)
        response = policy_rules[(state, match_intent(message))][1][number].format(*sharenames)
    # sector
    if match_intent(message) == 'sector':
        number, sharenames = respond_STO_SEAR(message)
        response = policy_rules[(state, match_intent(message))][1][number].format(*sharenames)
    # news     
    if match_intent(message) == 'news':
        response = policy_rules[(state, match_intent(message))][1].format(*get_news(entity)[0])
    # financial report     
    if match_intent(message) == 'finreport':
        response = policy_rules[(state, match_intent(message))][1].format(*get_latest_financials(entity))
        
    
    return new_state, response, pending_state



In [ ]:
# Stock information

# Define get_current_price()
def get_current_price(company):
    print("Company: ", company)   
    prices = Stock(company).get_price()
    return prices

# Define get_open_close_price()
def get_op_cl(company):
    print("Company: ", company)   
    opcl = Stock(company).get_open_close()
    return [company, opcl['open']['price'], opcl['close']['price'], opcl['high'], opcl['low']]

# Define get_previous_price()
def get_previous_price(company):
    print("Company: ", company)   
    prepri = Stock(company).get_previous()
    return [company, prepri['date'], prepri['open'], prepri['close'], prepri['high'], prepri['low'], prepri['volume']]

# Define get month price functon
def get_monthly_price(company):
    print("Company: ", company)   
    # get data from iexfinance 
    mon_pri_dics = Stock(company).get_historical_prices() 
    # create a new list
    mon_pri_data = []
    # change the data into the form that mpf.candlestick_ochl function can read
    for dic in mon_pri_dics:
        # get date open high low close volume
        list(dic.values())[0:6]
        # change date into number
        t = date2num(datetime.datetime.strptime(list(dic.values())[0], '%Y-%m-%d'))
        # change sequence into the form that mpf.candlestick_ochl function can read
        ope, high, low, close, vol = list(dic.values())[1:6]
         # change the data into the form that mpf.candlestick_ochl function can read
        data = [t, ope, close, high, low, vol]
        mon_pri_data.append(data)
    
    # draw the figure
    # set ax as the style of figure   
    fig, ax = plt.subplots(figsize = (10,5))
    plt.rcParams['savefig.dpi'] = 500
    # set parameters of mpf.candlestick_ochl: width = day; ax = ax sample; mon_pri_data = data color up = close > open; color down = open > close; alpha = transparent degree
    mpf.candlestick_ochl(ax, mon_pri_data, width = 0.3, colorup = 'g', colordown = 'r', alpha = 1.0)
    ax.set_title('Candlestick Chart of' + ' ' + company)
    plt.grid(True)
    # set price label
    plt.ylabel('Price')
    # set x axis as date
    ax.xaxis_date ()
    return plt.savefig('monthprice.png')

# Define get month abstract price
def get_monthly_abstract_price(company):
    print("Company: ", company)   
    # get data from iexfinance 
    mon_pri_dics = Stock(company).get_historical_prices() 
    # create a new list
    mon_pri_data = []
    # change the data into the form that mpf.candlestick_ochl function can read
    for dic in mon_pri_dics:
        # get date open high low close volume
        list(dic.values())[0:6]
        # change date into number
        t = date2num(datetime.datetime.strptime(list(dic.values())[0], '%Y-%m-%d'))
        # change sequence into the form that mpf.candlestick_ochl function can read
        ope, high, low, close, vol = list(dic.values())[1:6]
         # change the data into the form that mpf.candlestick_ochl function can read
        data = [t, ope, close, high, low, vol]
        mon_pri_data.append(data)
    
    # draw the figure
    # set ax as the style of figure   
    fig, ax = plt.subplots(figsize = (15,3.5))
    plt.rcParams['savefig.dpi'] = 500
    # set parameters of plot_day_summary_oclh: ax = ax sample; mon_pri_data = data color up = close > open; color down = open > close
    mpf.plot_day_summary_oclh(ax, mon_pri_data, colorup = 'g', colordown = 'r')
    ax.set_title('Candlestick Abstract Chart of' + ' ' + company)
    plt.grid(True)
    # set price label
    plt.ylabel('Price')
    # set x axis as date
    ax.xaxis_date ()
    return plt.savefig('monthabstractprice.png')

# Define get month price and volume functon
def get_monthly_volume_price(company):
    print("Company: ", company)   
    # get data from iexfinance 
    mon_pri_dics = Stock(company).get_historical_prices() 
    # create a new list
    mon_pri_data = []
    # change the data into the form that mpf.candlestick_ochl function can read
    for dic in mon_pri_dics:
        # get date open high low close volume
        list(dic.values())[0:6]
        # change date into number
        t = date2num(datetime.datetime.strptime(list(dic.values())[0], '%Y-%m-%d'))
        # change sequence into the form that mpf.candlestick_ochl function can read
        ope, high, low, close, vol = list(dic.values())[1:6]
        # change the data into the form that mpf.candlestick_ochl function can read
        data = [t, ope, close, high, low, vol]
        mon_pri_data.append(data)

    # change the data into the form that mpf.candlestick_ochl function can read
    datedata = [data[0]-0.25 for data in mon_pri_data]
    volumedata = [data[5] for data in mon_pri_data]
    # draw the figure of month price
    # set parameters of mpf.candlestick_ochl
    fig, (ax1, ax2) = plt.subplots(2, sharex = True, figsize = (15,8))
    plt.rcParams['savefig.dpi'] = 300
    mpf.candlestick_ochl(ax1, mon_pri_data, width = 0.6, colorup = 'g', colordown = 'r')
    ax1.set_title('Month price and Volume of' + ' ' + company)
    ax1.set_ylabel('Price')
    ax1.grid(True)
    ax1.xaxis_date()
    # draw the figure of volume
    plt.bar(datedata, volumedata, width = 0.5)
    ax2.set_ylabel('Volume')
    ax2.grid(True)
    return plt.savefig('monthvolumeprice.png')

#Define draw the figure of sector performance function
#get dataframe
def get_sector_perform():
    df = get_sector_performance(output_format='pandas')
    #set the size 
    plt.rcParams['figure.figsize'] = (16.5, 3.5)
    plt.rcParams['savefig.dpi'] = 300 
    plt.rcParams['figure.dpi'] = 300 
    #draw the barh figure
    df.iloc[2].plot(kind = 'barh')
    plt.xlabel("performance")
    plt.ylabel("sector")
    return plt.savefig('sectorperformance.png')

# Define timestamp to date Y-m-d H:M:S() in order to change the time in get quote()
def num_to_date(timestamp):
    dateArray = datetime.datetime.utcfromtimestamp(timestamp/1000)
    otherStyleTime = dateArray.strftime("%Y-%m-%d %H:%M:%S")
    return otherStyleTime

# Define get_quote() 
def get_quote(company):
    print("Company: ", company)   
    quote = Stock(company).get_quote()
    return [company, quote['companyName'], quote['primaryExchange'], quote['sector'], quote['open'], quote['close'], quote['high'], quote['low'], num_to_date(quote['openTime']), num_to_date(quote['closeTime']), quote['latestPrice'], quote['delayedPrice'], num_to_date(quote['delayedPriceTime']), quote['extendedPrice'], num_to_date(quote['extendedPriceTime']), quote['iexVolume'], quote['avgTotalVolume'], quote['week52High'], quote['week52Low']]

#Define get news function
def get_news(company):
    news = Stock(company).get_news()
    news_data = []
    for dic in news:
        list(dic.values())[0:5]
        date = list(dic.values())[0][0:10] +" " + list(dic.values())[0][11:19] 
        headline, source, url, summary = list(dic.values())[1:5]
        data = [date, headline, source, url, summary]
        news_data.append(data)
    return news_data

#Define get financial report function
def get_latest_financials(company):
    print("Company: ", company)
    financials = Stock(company).get_financials()[0]
    fin_report = []
    for i in list(financials.values()):
        fin_report.append(i)
    return fin_report

#Define get ipo function
def get_ipo_info():
    get_ipo_calendar()["viewData"][0:4]
    ipo_data = []
    for dic in get_ipo_calendar()["viewData"][0:5]:
        list(dic.values())[0:9]
        company = list(dic.values())[0][0:-1]
        symbol, price, shares, amount, flo, percent, market, expected = list(dic.values())[1:9]
        data = [company, symbol, price, shares, amount, flo, percent, market, expected]
        ipo_data.append(data)
    return ipo_data

# Define negated_ents()
def negated_ents(phrase, ent_vals):
    ents = [e for e in ent_vals if e in phrase]
    ends = sorted([phrase.index(e) + len(e) for e in ents])
    start = 0
    chunks = []
    for end in ends:
        chunks.append(phrase[start:end])
        start = end
    result = {}
    for chunk in chunks:
        for ent in ents:
            if ent in chunk:
                if "not" in chunk or "n't" in chunk:
                    result[ent] = False
                else:
                    result[ent] = True
    return result

# Define find_stocks()
def find_stocks(params, neg_params):
    query = 'SELECT * FROM stocks'
    if len(params) > 0 and len(neg_params) > 0:
        filters = ["{}=?".format(k) for k in params] + ["{}!=?".format(k) for k in neg_params]
        query += " WHERE " + " and ".join(filters)
    elif len(neg_params) > 0:
        filters = ["{}!=?".format(k) for k in neg_params]
        query += " WHERE " + " and ".join(filters)
    elif len(params) > 0:
        filters = ["{}=?".format(k) for k in params]
        query += " WHERE " + " and ".join(filters)
    
    t = tuple(dict(list(params.items()) + list(neg_params.items())).values())
    # open connection to DB
    conn = sqlite3.connect('stocks.db')
    # create a cursor
    c = conn.cursor()
    c.execute(query, t)
    return c.fetchall()

# Define the respond_STO_SEAR function
def respond_STO_SEAR(message):
    # Extract the entities
    entities = interpreter.parse(message)["entities"]
    ent_vals = [e["value"] for e in entities]
    # Look for negated entities
    negated = negated_ents(message, ent_vals)
    for ent in entities:
        if ent["value"] in negated and not negated[ent["value"]]:
            neg_params[ent["entity"]] = str(ent["value"])
        else:
            params[ent["entity"]] = str(ent["value"])
    # Find the hotels
    results = find_stocks(params, neg_params)
    names = [r[0] for r in results]
    n = min(len(results),3)
    # Return the correct response
    return n, names

In [ ]:
# Template
user_template = "User : {0}"
intent_template = "Intent : {0}"

# Initialize params and neg_params
params = {}
neg_params = {}

# Define global state
global state
state = MENU

global pending
pending = None

# Import wxpy
from wxpy import *

# Create the chatbot
bot = Bot()

# Find the subject of chat
my_friend = bot.friends().search('可')[0]

# Define reply_my_friend()
@bot.register(my_friend)
def reply_my_friend(msg):  
    global state 
    global pending
    print(user_template.format(msg.text))
    print(intent_template.format(match_intent(msg.text)))
    state, pending = send_message(state, pending, msg.text)
    return None

#Check the current registered function 
bot.registered
